In [24]:
import thinkplot
import thinkstats2
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import scipy.stats as ss
import math
import random
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

<h1>Regression End to End with Categorical Varaibles</h1>

We can walk through a realistic example now. 

The target is the price, in the variable "buy_price"

In [25]:
df = pd.read_csv("houses_Madrid.csv")
df = df.drop(columns={"Unnamed: 0","is_buy_price_known","id","title","is_rent_price_known","neighborhood_id","house_type_id","rent_price"})
df.head()

,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,latitude,longitude,raw_address,...,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
0,"San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,NaN,NaN,"Calle de Godella, 64",...,D,False,NaN,NaN,NaN,NaN,False,True,False,False
1,"Los Ángeles, Madrid",70.0,NaN,3,1.0,NaN,NaN,NaN,NaN,Calle de la del Manojo de Rosas,...,en trámite,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"San Andrés, Madrid",94.0,54.0,2,2.0,NaN,NaN,NaN,NaN,"Calle del Talco, 68",...,no indicado,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"San Andrés, Madrid",64.0,NaN,2,1.0,NaN,NaN,NaN,NaN,Calle Pedro Jiménez,...,en trámite,False,NaN,NaN,NaN,NaN,False,False,True,False
4,"Los Rosales, Madrid",108.0,90.0,2,2.0,NaN,NaN,NaN,NaN,Carretera de Villaverde a Vallecas,...,en trámite,True,NaN,NaN,True,0.0,True,True,True,True


In [26]:
#Describe
df.describe()

,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,latitude,longitude,portal,door,rent_price_by_area,buy_price,buy_price_by_area,built_year,are_pets_allowed,is_furnished,is_kitchen_equipped,has_private_parking,has_public_parking,parking_price
count,21616.000000,8228.000000,21742.000000,21726.000000,1437.000000,1432.000000,0.0,0.0,0.0,0.0,0.0,2.174200e+04,21742.000000,10000.000000,0.0,0.0,0.0,0.0,0.0,7719.000000
mean,146.920892,103.458192,3.005749,2.091687,3.128740,241.692737,NaN,NaN,NaN,NaN,NaN,6.537356e+05,4020.523871,1970.046400,NaN,NaN,NaN,NaN,NaN,2658.000518
std,134.181865,88.259192,1.510497,1.406992,0.907713,247.484853,NaN,NaN,NaN,NaN,NaN,7.820821e+05,1908.418774,69.386705,NaN,NaN,NaN,NaN,NaN,13360.966258
min,13.000000,1.000000,0.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,3.600000e+04,447.000000,1723.000000,NaN,NaN,NaN,NaN,NaN,0.000000
25%,70.000000,59.000000,2.000000,1.000000,2.000000,2.000000,NaN,NaN,NaN,NaN,NaN,1.980000e+05,2551.000000,1957.000000,NaN,NaN,NaN,NaN,NaN,0.000000
50%,100.000000,79.000000,3.000000,2.000000,3.000000,232.000000,NaN,NaN,NaN,NaN,NaN,3.750000e+05,3720.000000,1970.000000,NaN,NaN,NaN,NaN,NaN,0.000000
75%,162.000000,113.000000,4.000000,2.000000,4.000000,354.000000,NaN,NaN,NaN,NaN,NaN,7.636000e+05,5000.000000,1994.000000,NaN,NaN,NaN,NaN,NaN,0.000000
max,999.000000,998.000000,24.000000,16.000000,7.000000,997.000000,NaN,NaN,NaN,NaN,NaN,8.800000e+06,18889.000000,8170.000000,NaN,NaN,NaN,NaN,NaN,600000.000000


In [27]:
#Check for nulls 
df.isnull().sum().sort_values(ascending=False)

has_private_parking             21742
door                            21742
are_pets_allowed                21742
is_furnished                    21742
latitude                        21742
longitude                       21742
is_kitchen_equipped             21742
rent_price_by_area              21742
portal                          21742
has_public_parking              21742
sq_mt_allotment                 20310
n_floors                        20305
has_garden                      20186
has_balcony                     18421
has_green_zones                 17685
is_accessible                   17668
has_pool                        16571
street_number                   15442
has_storage_room                14044
is_parking_included_in_price    14023
parking_price                   14023
sq_mt_useful                    13514
has_terrace                     12194
built_year                      11742
has_ac                          10531
is_orientation_north            10384
is_orientati

We have lots of columns, and lots of nulls. I'm going to get rid of a few right now, starting with things that are very sparse. 

In [28]:
limitPer = len(df) * .80
df = df.dropna(thresh=limitPer, axis=1)
df.head()

,subtitle,sq_mt_built,n_rooms,n_bathrooms,is_exact_address_hidden,floor,is_floor_under,operation,buy_price,buy_price_by_area,is_renewal_needed,is_new_development,has_lift,is_exterior,energy_certificate,has_parking
0,"San Cristóbal, Madrid",64.0,2,1.0,False,3,False,sale,85000,1328,False,False,False,True,D,False
1,"Los Ángeles, Madrid",70.0,3,1.0,True,4,False,sale,129900,1856,True,False,True,True,en trámite,False
2,"San Andrés, Madrid",94.0,2,2.0,False,1,False,sale,144247,1535,False,False,True,True,no indicado,False
3,"San Andrés, Madrid",64.0,2,1.0,True,Bajo,True,sale,109900,1717,False,False,True,True,en trámite,False
4,"Los Rosales, Madrid",108.0,2,2.0,True,4,False,sale,260000,2407,False,False,True,True,en trámite,True


Ok, we had a tonne of data there, now we have a moderate amount and we can do some predicting. First, we should explore...

In [ ]:
#Get an idea of distributions and correlations.
sns.pairplot(df)